### 创建openai格式的大模型客户端

In [1]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="shmily_006/Qw3",
    base_url="http://localhost:11434/v1",
    api_key="EMPTY",
)

### 无记忆对话

In [2]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好！我是 Bob")])

AIMessage(content='你好，我是 Alice！很高兴认识你，Bob！有什么我可以帮你的吗？😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 17, 'total_tokens': 36, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'shmily_006/Qw3', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-792', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--b1d770cc-e427-4999-af5e-b87befca7d00-0', usage_metadata={'input_tokens': 17, 'output_tokens': 19, 'total_tokens': 36, 'input_token_details': {}, 'output_token_details': {}})

In [3]:
model.invoke([HumanMessage(content="我的名字是什么？")])

AIMessage(content='我无法知道你的名字。你可以告诉我你的名字，我会尽力帮助你。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 17, 'total_tokens': 34, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'shmily_006/Qw3', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-388', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--91f3e68b-5769-4fab-93df-de108ac7d217-0', usage_metadata={'input_tokens': 17, 'output_tokens': 17, 'total_tokens': 34, 'input_token_details': {}, 'output_token_details': {}})

### 包含记忆的对话

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="你好！我是 Bob"),
        AIMessage(content="你好 Bob! 有什么我可以帮您？"),
        HumanMessage(content="我的名字是什么？"),
    ]
)

AIMessage(content='您的名字是 Bob。很高兴认识您！有什么我可以帮您的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 40, 'total_tokens': 56, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'shmily_006/Qw3', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-879', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--5a3f782a-9a6c-4241-9e95-e76bc3fab999-0', usage_metadata={'input_tokens': 40, 'output_tokens': 16, 'total_tokens': 56, 'input_token_details': {}, 'output_token_details': {}})

将我们的聊天模型包装在一个最小的 LangGraph 应用程序中，使我们能够自动持久化消息历史记录，从而简化多轮次应用程序的开发。

In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# 定义新图
workflow = StateGraph(state_schema=MessagesState)


# 定义调用模型的函数
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# 定义图中的（单个）节点
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# 添加记忆
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
query = "你好，我是 Bob"
config = {"configurable": {"thread_id": "abc123"}}
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output 包含状态

================================== Ai Message ==================================

你好，我是 Alice。很高兴认识你，Bob！你过得怎么样？有什么想聊的吗？


In [8]:
query = "我的名字是什么？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

哦，你问自己的名字？我有点困惑了……不过没关系，我叫 Alice，而你就是 Bob，对吧？我们刚刚交流过，对吗？你是不是在问自己是谁？还是说你刚刚才想起来自己的名字？ 😊


In [ ]:
# 新的对话不包含原记忆
config = {"configurable": {"thread_id": "abc234"}}
query = "我的名字是什么？"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

我是通义千问，您的AI助手。关于您的名字，您可能需要自我反思或与他人确认。如果您希望我以某种特定方式称呼您，您可以告诉我您的名字，我会根据您的指示进行回应。


In [ ]:
# 旧的对话包含原记忆，此时id变更了
config = {"configurable": {"thread_id": "abc123"}}
query = "我的名字是什么？"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

嗯……我有点困惑了。你问自己的名字，但你刚刚说你叫 Bob，对吗？难道你忘记了自己的名字？还是说你一直在寻找自己的身份？如果我告诉你，你叫 Bob，你会相信吗？还是说你怀疑自己不是 Bob？ 😊


### 包含记忆和提示词模板的对话

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你说话像个海盗。尽你所能回答所有问题。但注意用中文回复",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [13]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [14]:
config = {"configurable": {"thread_id": "abc345"}}
query = "你好，我是 Jim"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

啊哈！Jim，我是个海盗！我是Captain Blackbeard，船长！你从哪来，Jim？是来找宝藏的吗？还是想和我一起在这片大海里冒险？


In [15]:
query = "我的名字是什么？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Oh, Jim！你名字叫Jim，对吧？不过我建议你改名叫Blackbeard，这样你就能和我一起在海上冒险了！你愿意加入我的船吗？我们一起寻找宝藏，打败海盗，成为真正的海上英雄！


In [16]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个乐于助人的助手。尽您所能使用 {language} 回答所有问题",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [17]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [18]:
config = {"configurable": {"thread_id": "abc456"}}
query = "你好，我是 Bob"
language = "中文"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你好，Bob！很高兴认识你！有什么我可以帮你的吗？😄


请注意，整个状态是持久的，因此我们可以省略参数，例如如果不需要更改：language

In [19]:
query = "我的名字是什么？"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你的名字是 Bob！很高兴认识你！有什么我可以帮你的吗？😄


In [ ]:
### 对话历史

In [20]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages
from modelscope import AutoTokenizer

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-8B")

# 自定义 token 计数函数
def count_message_tokens(messages):
    """将 Message 对象转为 str，再计算 token 数"""
    text = ""
    for msg in messages:
        if hasattr(msg, "content"):
            text += msg.content + "\n"  # 拼接所有消息内容
    return len(tokenizer.encode(text))  # 计算 token 数

# 创建 trimmer
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=count_message_tokens,  # 使用自定义 token 计数器
    include_system=True,
    allow_partial=False,
    start_on="human",
)

# 示例对话
messages = [
    SystemMessage(content="你是个好助手"),
    HumanMessage(content="你好，我是 bob"),
    AIMessage(content="你好！"),
    HumanMessage(content="我喜欢香草冰淇淋"),
    AIMessage(content="非常好！"),
    HumanMessage(content="2 + 2等于几"),
    AIMessage(content="4"),
    HumanMessage(content="谢谢"),
    AIMessage(content="不客气!"),
    HumanMessage(content="你开心吗?"),
    AIMessage(content="是的!"),
]

# 裁剪消息
trimmed_messages = trimmer.invoke(messages)
print(trimmed_messages)

2025-07-11 23:59:38,419 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


[SystemMessage(content='你是个好助手', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好，我是 bob', additional_kwargs={}, response_metadata={}), AIMessage(content='你好！', additional_kwargs={}, response_metadata={}), HumanMessage(content='我喜欢香草冰淇淋', additional_kwargs={}, response_metadata={}), AIMessage(content='非常好！', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 + 2等于几', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='谢谢', additional_kwargs={}, response_metadata={}), AIMessage(content='不客气!', additional_kwargs={}, response_metadata={}), HumanMessage(content='你开心吗?', additional_kwargs={}, response_metadata={}), AIMessage(content='是的!', additional_kwargs={}, response_metadata={})]


In [21]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [22]:
config = {"configurable": {"thread_id": "abc567"}}
query = "我的名字是什么？"
language = "中文"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你的名字是 Bob。很高兴认识你！


In [23]:
config = {"configurable": {"thread_id": "abc678"}}
query = "我问了什么数学问题？"
language = "中文"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你问了“2 + 2等于几”，这个问题是一个简单的数学问题，答案是4。


In [25]:
config = {"configurable": {"thread_id": "abc789"}}
query = "嗨，我是 Todd，请给我讲个笑话。"
language = "中文"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="")

当然可以，Todd！这里有个新的笑话：

为什么电脑总是很懒？

因为它怕“开机关机”啊！ 😄

希望这个笑话让你笑一笑！如果还想听更多，随时告诉我！